08/04/2020

Melanocytes vocab

In [1]:
# basic packages
import os, glob
import pandas as pd
import numpy as np; np.random.seed(0)
import itertools
from collections import Counter, defaultdict
import time
# Import tools needed for visualization
import seaborn as sns; sns.set()
import matplotlib
import matplotlib.pyplot as plt

In [2]:
MAX_DIST=135


In [5]:
#get tfs
tf_df = pd.read_csv('../data/external/transcription_factor_info_063020.csv')
TFS = sorted(set(tf_df[tf_df['cell_type']=='Melanocytes']["tf"]))
print(len(TFS))
print(TFS)


28
['ATF4', 'CERS2', 'CSDE1', 'EDF1', 'EIF3K', 'ETS1', 'EWSR1', 'H1F0', 'HMG20B', 'HMGB1', 'IFI16', 'IRF4', 'LEF1', 'MITF', 'NFE2L1', 'PAX3', 'PHB2', 'PLXNC1', 'RB1', 'SNAI2', 'SOX10', 'SREBF2', 'TCEAL8', 'TFAP2A', 'USF2', 'YBX1', 'YY1', 'ZFP106']


In [30]:
glob.glob('../data/processed/fig4_modelling/*')

['../data/processed/fig4_modelling/KRT_ex',
 '../data/processed/fig4_modelling/pro_tf_only_random_feature_importance.csv',
 '../data/processed/fig4_modelling/comb_tf_only_allcrms_feature_importance.csv',
 '../data/processed/fig4_modelling/dist_tf_only_df_confmat_ppv.pdf',
 '../data/processed/fig4_modelling/dimred_pca12.png',
 '../data/processed/fig4_modelling/dimred_umap.png',
 '../data/processed/fig4_modelling/dist_tf_only_auc_curve_values.csv',
 '../data/processed/fig4_modelling/comb_all_feature_importance.csv',
 '../data/processed/fig4_modelling/pro_dist_sum_all_auc_tissue.csv',
 '../data/processed/fig4_modelling/stability_df_by_tissue_skin_multi.eps',
 '../data/processed/fig4_modelling/dist_tf_only_random_auc_curve_values.csv',
 '../data/processed/fig4_modelling/pro_tf_only_random_eval_df.csv',
 '../data/processed/fig4_modelling/prom_tf_only_df_confmat_ppv.pdf',
 '../data/processed/fig4_modelling/dist_tf_only_feature_importance.csv',
 '../data/processed/fig4_modelling/Icon\r',
 '..

In [28]:
vocab_dir = '../data/processed/fig4_modelling/tf_tf_pairs/'
glob.glob(vocab_dir+'*vocab*')

['../data/processed/fig4_modelling/tf_tf_pairs/expr_GDSD6_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Pancreas_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Colon_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Airway_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Ovarian_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Astrocytes_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Thyroid_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Esophageal_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/unique_alltissue_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Renal_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/unique_alltissue_vocab_info.xlsx',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Melanocytes_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Uterine_voc

In [7]:
glob.glob('../data/processed/fig4_modelling/tf_tf_pairs_stability/'+'*vocab*')

['../data/processed/fig4_modelling/tf_tf_pairs_stability/HMEC_vocab_group.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs_stability/GDSD6_vocab.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs_stability/Prostate_vocab.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs_stability/Ovarian_vocab_group.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs_stability/~$GDSD6_vocab.xlsx',
 '../data/processed/fig4_modelling/tf_tf_pairs_stability/Airway_vocab_group.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs_stability/Prostate_vocab_group.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs_stability/GDSD6_vocab.xlsx',
 '../data/processed/fig4_modelling/tf_tf_pairs_stability/GM12878_vocab_group.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs_stability/Uterine_vocab_group.csv']

['../data/processed/fig4_modelling/tf_tf_pairs/expr_GDSD6_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Pancreas_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Colon_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Airway_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Ovarian_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Astrocytes_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Thyroid_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Esophageal_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/unique_alltissue_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Renal_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/unique_alltissue_vocab_info.xlsx',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Melanocytes_vocab_info.csv',
 '../data/processed/fig4_modelling/tf_tf_pairs/expr_Uterine_voc

In [11]:
def get_other_vocab_word(row, next_row):
    if row['vocab']!=next_row['vocab']:
        return False
    
    vocab_word = set(row['tf'])
    vocab_set = set(row['vocab'].split('::'))
    other_vocab = list(vocab_set -  vocab_word)[0]
    return other_vocab == next_row['tf']

def check_distance(row,next_row,max_dist=MAX_DIST):
    if row['chr_m']==next_row['chr_m']:
        if row['stop_m']<next_row['start_m']:
            tot_dist = next_row['stop_m'] - row['start_m']
            btn_dist = next_row['start_m'] - row['stop_m']
            return (tot_dist < max_dist), tot_dist, btn_dist
    return False,-1,-1

def get_hits(vocab_file,tfs=TFS):
    print('**** reading', vocab_file)
    vocab_df = pd.read_csv(vocab_file)
    print(vocab_df.shape)
    idx = 0
    idx_hits = 0
    results_dict = {}
    while idx < (vocab_df.shape[0]-1):
        # look at next 
        row = vocab_df.iloc[idx,:]
        next_row = vocab_df.iloc[idx+1,:]

        check_vocab_pair = get_other_vocab_word(row,next_row)
        check_dist,tot_dist, btn_dist = check_distance(row, next_row)
        check_tissue_tf = (row['tf'] in tfs) | (next_row['tf'] in tfs)
        if (check_dist and check_vocab_pair):
    #         print('hi',idx)
    #         print(row)
    #         print(next_row)

            results_dict[idx_hits] = {'vocab_pair':row['vocab'],'tot_dist':tot_dist,'btn_dist':btn_dist,
                                      'chr':row['chr'],'start':row['start_m'],'stop':next_row['stop_m'], 
                                      'vocab1':row['tf'],'vocab1_start':row['start_m'], 'vocab1_stop': row['stop_m'],
                                      'vocab2':next_row['tf'],'vocab2_start':next_row['start_m'], 'vocab2_stop': next_row['stop_m'],
                                      'genes':row['genes'],'num_genes':len(row['genes'].split('|')), 'tissue':row['tissue'],
                                     'check_tissuetf':check_tissue_tf}
            idx_hits+=1

        idx+=1
    print('num_hits',idx_hits)
    results_df = pd.DataFrame.from_dict(results_dict, orient='index')
    return results_df
    
def filter_results(results_df,min_hits_per_vocab=10):
    print('shae', results_df.shape)
    vocab_counts = results_df.vocab_pair.value_counts()
    print('original num vocab',vocab_counts.shape[0])
    vocab_to_include = vocab_counts[vocab_counts>min_hits_per_vocab].index.values
    print('filt num vocab',vocab_to_include.shape[0])
    
    results_df_filt = results_df[results_df.vocab_pair.isin(vocab_to_include)]
    return results_df_filt


In [18]:
%%time
# filtered so that at least 10 genomic instances of vocab pair in significantly found in expressed genes 
results_expr = get_hits('../data/processed/fig4_modelling/tf_tf_pairs/expr_Melanocytes_vocab_info.csv')
# results_stability = get_hits('../data/processed/fig4_modelling/tf_tf_pairs_stability/GDSD6_vocab.csv')


**** reading ../data/processed/fig4_modelling/tf_tf_pairs/expr_Melanocytes_vocab_info.csv
(16151, 15)
num_hits 4446
CPU times: user 6.46 s, sys: 19.8 ms, total: 6.48 s
Wall time: 6.49 s


In [26]:
motifs = sorted(set(list(results_expr.vocab1.unique())+list(results_expr.vocab2.unique())))
print(len(motifs))
print(motifs)

61
['ARNT2', 'BATF3', 'CPEB1', 'CREB3L1', 'E2F2', 'EBF1', 'EN2', 'ETV1', 'ETV2', 'FOXD3', 'FOXF2', 'FOXL1', 'GLI3', 'GLIS3', 'HEY1', 'HOXC10', 'HOXC11', 'HOXC6', 'HOXC8', 'HOXD3', 'HOXD8', 'HSF4', 'ID4', 'IRF4', 'KLF12', 'KLF15', 'KLF9', 'LEF1', 'MAF', 'MEF2C', 'MEIS1', 'MEIS3', 'MYOD1', 'NFATC1', 'NFATC2', 'NFATC4', 'NR4A3', 'PAX3', 'POU3F1', 'PRRX1', 'RARB', 'SALL4', 'SOX10', 'SOX11', 'SOX5', 'SP2', 'TBX2', 'TFAP2B', 'TFEB', 'TWIST1', 'ZFP82', 'ZNF18', 'ZNF219', 'ZNF324', 'ZNF41', 'ZNF418', 'ZNF423', 'ZNF436', 'ZNF449', 'ZNF589', 'ZNF708']


In [27]:
print(results_expr.vocab_pair.unique().shape)
results_expr.vocab_pair.unique()

(269,)


array(['ALX1::EN2', 'ALX1::PRRX1', 'ALX1::ZNF589', 'ARNT2::EBF1',
       'ARNT2::EN2', 'ARNT2::ETV2', 'ARNT2::FOXL1', 'ARNT2::HOXC10',
       'ARNT2::HOXD8', 'ARNT2::IRF4', 'ARNT2::KLF15', 'ARNT2::MEIS1',
       'ARNT2::MYOD1', 'ARNT2::NFATC1', 'ARNT2::POU3F1', 'ARNT2::SALL4',
       'ARNT2::SOX10', 'ARNT2::TFAP2B', 'ARNT2::TFEB', 'ARNT2::ZNF324',
       'ARNT2::ZNF41', 'BATF3::KLF15', 'BATF3::MYOD1', 'CPEB1::ETV2',
       'CPEB1::FOXF2', 'CPEB1::KLF15', 'CPEB1::NFATC4', 'CPEB1::PRRX1',
       'CPEB1::SOX11', 'CPEB1::SP2', 'CPEB1::ZNF589', 'CREB3L1::KLF15',
       'CREB3L1::MYOD1', 'CREB3L1::TFAP2B', 'E2F2::MEIS1', 'E2F2::MYOD1',
       'EBF1::GLIS3', 'EBF1::KLF15', 'EBF1::KLF9', 'EBF1::PRRX1',
       'EBF1::TFAP2B', 'EBF1::ZNF219', 'EBF1::ZNF324', 'EN2::ETV2',
       'EN2::FOXD3', 'EN2::GLI3', 'EN2::HOXC11', 'EN2::HOXD3',
       'EN2::HOXD8', 'EN2::IRF4', 'EN2::KLF12', 'EN2::KLF15',
       'EN2::MYOD1', 'EN2::NFATC4', 'EN2::PAX3', 'EN2::PRRX1',
       'EN2::RARB', 'EN2::SALL4', 'EN2::

1. filter expression enriched vocab words if then have at least 10 genomic instances

In [16]:
results_expr = filter_results(results_expr,min_hits_per_vocab=10)
results_expr.vocab_pair.unique()

shae (3901, 16)
original num vocab 59
filt num vocab 59


array(['ARNT2::EN2', 'ARNT2::KLF15', 'ARNT2::SALL4', 'ARNT2::TFAP2B',
       'CPEB1::PRRX1', 'CPEB1::ZNF589', 'EN2::KLF12', 'EN2::NFATC4',
       'EN2::PRRX1', 'EN2::ZNF589', 'ETV2::KLF12', 'ETV2::NFATC4',
       'ETV2::PRRX1', 'ETV2::SP2', 'ETV2::ZNF219', 'ETV2::ZNF589',
       'FOXD3::PRRX1', 'FOXD3::ZNF589', 'FOXF2::PRRX1', 'FOXL1::PRRX1',
       'HOXC10::PRRX1', 'HOXC10::ZNF589', 'HOXC6::PRRX1', 'HOXC8::PRRX1',
       'HOXC8::ZNF589', 'HOXD3::PRRX1', 'HOXD3::SP2', 'HOXD3::ZNF219',
       'HOXD8::PRRX1', 'IRF4::KLF12', 'IRF4::PRRX1', 'KLF12::KLF15',
       'KLF12::SALL4', 'KLF12::TFAP2B', 'KLF15::PRRX1', 'KLF15::SALL4',
       'KLF15::TFAP2B', 'KLF15::ZNF219', 'KLF15::ZNF41', 'KLF15::ZNF589',
       'KLF9::TFAP2B', 'MEIS1::PRRX1', 'NFATC1::PRRX1', 'PAX3::PRRX1',
       'PAX3::ZNF589', 'POU3F1::PRRX1', 'POU3F1::ZNF589', 'PRRX1::SALL4',
       'PRRX1::SOX10', 'PRRX1::TFAP2B', 'PRRX1::ZNF324', 'PRRX1::ZNF41',
       'SALL4::ZNF219', 'SALL4::ZNF589', 'SP2::TFAP2B', 'SP2::ZNF324',
      

2. filter expr vocab words based on whether there is they are annotated for skin

In [109]:
results_expr[['vocab_pair','check_tissuetf']].drop_duplicates().check_tissuetf.value_counts()

False    71
True     59
Name: check_tissuetf, dtype: int64

59/(59+71) (~40%) of vocab words have at least annotated skin motif

In [112]:
# overall tf vocab words
print(results_expr.vocab_pair.unique())

['ARID5B::CREB5' 'ARID5B::DLX2' 'ARID5B::ELF5' 'ARID5B::ID4'
 'ARID5B::KLF5' 'ASCL2::KLF4' 'ASCL2::KLF5' 'ATF2::BARX2' 'ATF4::BARX2'
 'BARX2::CEBPB' 'BARX2::CEBPG' 'BARX2::CREB5' 'BARX2::CREM' 'BARX2::DBP'
 'BARX2::DDIT3' 'BARX2::EGR2' 'BARX2::EGR3' 'BARX2::ELF5' 'BARX2::GATA3'
 'BARX2::HBP1' 'BARX2::HOXA10' 'BARX2::HOXA5' 'BARX2::ID4' 'BARX2::MAFG'
 'BARX2::MEIS2' 'BARX2::NFIC' 'BARX2::NFIL3' 'BARX2::POU2F1'
 'BARX2::SALL4' 'BARX2::SMARCA1' 'BARX2::SOX7' 'BARX2::SREBF2'
 'BARX2::STAT3' 'BARX2::TCF7L2' 'BARX2::TEAD3' 'BARX2::UBP1'
 'BARX2::ZFHX3' 'CEBPG::NFATC4' 'CREB1::NFIA' 'CREB1::SP2' 'CREB1::UBP1'
 'CREB5::NFATC4' 'CREB5::NFIA' 'CREB5::SP2' 'CREB5::UBP1' 'CREM::NFATC4'
 'CREM::SP2' 'DDIT3::NFATC4' 'EGR2::ELF5' 'EGR2::EPAS1' 'EGR2::HOXA10'
 'EGR2::ID4' 'EGR2::KLF4' 'EGR2::KLF5' 'EGR2::KLF9' 'EGR2::MAF'
 'EGR2::MAFG' 'EGR2::NFE2L1' 'EGR2::NFIA' 'EGR2::NFIC' 'EGR2::SMAD3'
 'EGR2::TP63' 'EGR2::ZNF335' 'EGR2::ZNF467' 'EGR2::ZNF554' 'EGR3::KLF4'
 'EGR3::KLF5' 'EGR3::KLF9' 'EGR3::NFIA' '

In [106]:
# filter for annotated vocab in at least 1 vocab pair
results_expr_tissue = results_expr[results_expr.check_tissuetf]
print('total vocab:',results_expr.vocab_pair.unique().shape[0],'tissue annon vocab:', results_expr_tissue.vocab_pair.unique().shape[0])
print(results_expr_tissue.vocab_pair.unique())

total vocab: 116 tissue annon vocab: 59
['ARID5B::ELF5' 'ARID5B::KLF5' 'ASCL2::KLF4' 'ASCL2::KLF5' 'ATF2::BARX2'
 'ATF4::BARX2' 'BARX2::CEBPB' 'BARX2::CEBPG' 'BARX2::CREB5' 'BARX2::CREM'
 'BARX2::DBP' 'BARX2::DDIT3' 'BARX2::EGR2' 'BARX2::EGR3' 'BARX2::ELF5'
 'BARX2::GATA3' 'BARX2::HBP1' 'BARX2::HOXA10' 'BARX2::HOXA5' 'BARX2::ID4'
 'BARX2::MAFG' 'BARX2::MEIS2' 'BARX2::NFIC' 'BARX2::NFIL3' 'BARX2::POU2F1'
 'BARX2::SALL4' 'BARX2::SMARCA1' 'BARX2::SOX7' 'BARX2::SREBF2'
 'BARX2::STAT3' 'BARX2::TCF7L2' 'BARX2::TEAD3' 'BARX2::UBP1'
 'BARX2::ZFHX3' 'CEBPG::NFATC4' 'EGR2::ELF5' 'EGR2::KLF4' 'EGR2::KLF5'
 'EGR2::MAF' 'EGR2::TP63' 'EGR3::KLF4' 'EGR3::KLF5' 'ELF5::HMGA1'
 'ELF5::KLF4' 'ELF5::KLF5' 'ELF5::KLF9' 'ELF5::NFATC4' 'ELF5::NFIA'
 'ELF5::SP2' 'ELF5::UBP1' 'ID4::KLF4' 'ID4::RORA' 'KLF4::SREBF2'
 'KLF5::MAFG' 'KLF5::NFIC' 'KLF5::SALL4' 'KLF5::SREBF2' 'KLF5::UBP1'
 'RORA::ZNF467']


In [105]:
# filter both are found in esults_expr_tissue[results_expr_tissue.vocab1.isin(TFS) & results_expr_tissue.vocab2.isin(TFS)]

,vocab_pair,tot_dist,btn_dist,chr,start,stop,vocab1,vocab1_start,vocab1_stop,vocab2,vocab2_start,vocab2_stop,genes,num_genes,tissue,check_tissuetf
133,ARID5B::KLF5,64,42,chr1,6474720,6474784,KLF5,6474720,6474731,KLF5,6474773,6474784,ACVR1B|AP2S1|ATG9A|CCDC69|CUX1|DDX5|EMD|ESYT2|...,19,GDSD6,True
135,ARID5B::KLF5,133,111,chr1,152009415,152009548,KLF5,152009415,152009426,KLF5,152009537,152009548,ACVR1B|AP2S1|ATG9A|CCDC69|CUX1|DDX5|EMD|ESYT2|...,19,GDSD6,True
136,ARID5B::KLF5,80,58,chr1,154155146,154155226,KLF5,154155146,154155157,KLF5,154155215,154155226,ACVR1B|AP2S1|ATG9A|CCDC69|CUX1|DDX5|EMD|ESYT2|...,19,GDSD6,True
137,ARID5B::KLF5,112,90,chr1,154155215,154155327,KLF5,154155215,154155226,KLF5,154155316,154155327,ACVR1B|AP2S1|ATG9A|CCDC69|CUX1|DDX5|EMD|ESYT2|...,19,GDSD6,True
139,ARID5B::KLF5,44,22,chr12,6472517,6472561,KLF5,6472517,6472528,KLF5,6472550,6472561,ACVR1B|AP2S1|ATG9A|CCDC69|CUX1|DDX5|EMD|ESYT2|...,19,GDSD6,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5140,RORA::ZNF467,43,25,chr12,6472521,6472564,RORA,6472521,6472530,RORA,6472555,6472564,ABCA7|ACTN4|EHD1|EML3|EPHA2|HSPB1|LTBR|MGAT4B|...,14,GDSD6,True
5141,RORA::ZNF467,46,28,chr12,6472901,6472947,RORA,6472901,6472910,RORA,6472938,6472947,ABCA7|ACTN4|EHD1|EML3|EPHA2|HSPB1|LTBR|MGAT4B|...,14,GDSD6,True
5143,RORA::ZNF467,27,9,chr19,39127760,39127787,RORA,39127760,39127769,RORA,39127778,39127787,ABCA7|ACTN4|EHD1|EML3|EPHA2|HSPB1|LTBR|MGAT4B|...,14,GDSD6,True
5145,RORA::ZNF467,45,27,chr5,179222376,179222421,RORA,179222376,179222385,RORA,179222412,179222421,ABCA7|ACTN4|EHD1|EML3|EPHA2|HSPB1|LTBR|MGAT4B|...,14,GDSD6,True


In [99]:
results_expr_tissue

,vocab_pair,tot_dist,btn_dist,chr,start,stop,vocab1,vocab1_start,vocab1_stop,vocab2,vocab2_start,vocab2_stop,genes,num_genes,tissue,check_tissuetf
63,ARID5B::ELF5,25,12,chr1,154155670,154155695,ELF5,154155670,154155679,ARID5B,154155691,154155695,BCO2|DDX5|DYNC1LI2|GRB2|GTF3C6|SLC9A1|STK17A|TPM3,8,GDSD6,True
68,ARID5B::ELF5,93,80,chr11,112034838,112034931,ELF5,112034838,112034847,ARID5B,112034927,112034931,BCO2|DDX5|DYNC1LI2|GRB2|GTF3C6|SLC9A1|STK17A|TPM3,8,GDSD6,True
69,ARID5B::ELF5,34,21,chr16,66774484,66774518,ELF5,66774484,66774493,ARID5B,66774514,66774518,BCO2|DDX5|DYNC1LI2|GRB2|GTF3C6|SLC9A1|STK17A|TPM3,8,GDSD6,True
73,ARID5B::ELF5,29,16,chr17,62493349,62493378,ELF5,62493349,62493358,ARID5B,62493374,62493378,BCO2|DDX5|DYNC1LI2|GRB2|GTF3C6|SLC9A1|STK17A|TPM3,8,GDSD6,True
75,ARID5B::ELF5,41,28,chr17,62493416,62493457,ELF5,62493416,62493425,ARID5B,62493453,62493457,BCO2|DDX5|DYNC1LI2|GRB2|GTF3C6|SLC9A1|STK17A|TPM3,8,GDSD6,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5147,RORA::ZNF467,90,60,chr5,179223104,179223194,ZNF467,179223104,179223125,RORA,179223185,179223194,ABCA7|ACTN4|EHD1|EML3|EPHA2|HSPB1|LTBR|MGAT4B|...,14,GDSD6,True
5148,RORA::ZNF467,45,27,chr5,179222376,179222421,RORA,179222376,179222385,RORA,179222412,179222421,ABCA7|ACTN4|EHD1|EML3|EPHA2|HSPB1|LTBR|MGAT4B|...,14,GDSD6,True
5149,RORA::ZNF467,74,44,chr5,179222802,179222876,ZNF467,179222802,179222823,RORA,179222867,179222876,ABCA7|ACTN4|EHD1|EML3|EPHA2|HSPB1|LTBR|MGAT4B|...,14,GDSD6,True
5150,RORA::ZNF467,90,60,chr5,179223104,179223194,ZNF467,179223104,179223125,RORA,179223185,179223194,ABCA7|ACTN4|EHD1|EML3|EPHA2|HSPB1|LTBR|MGAT4B|...,14,GDSD6,True


In [101]:
results_expr.vocab_pair.unique()

array(['ARID5B::CREB5', 'ARID5B::DLX2', 'ARID5B::ELF5', 'ARID5B::ID4',
       'ARID5B::KLF5', 'ASCL2::KLF4', 'ASCL2::KLF5', 'ATF2::BARX2',
       'ATF4::BARX2', 'BARX2::CEBPB', 'BARX2::CEBPG', 'BARX2::CREB5',
       'BARX2::CREM', 'BARX2::DBP', 'BARX2::DDIT3', 'BARX2::EGR2',
       'BARX2::EGR3', 'BARX2::ELF5', 'BARX2::GATA3', 'BARX2::HBP1',
       'BARX2::HOXA10', 'BARX2::HOXA5', 'BARX2::ID4', 'BARX2::MAFG',
       'BARX2::MEIS2', 'BARX2::NFIC', 'BARX2::NFIL3', 'BARX2::POU2F1',
       'BARX2::SALL4', 'BARX2::SMARCA1', 'BARX2::SOX7', 'BARX2::SREBF2',
       'BARX2::STAT3', 'BARX2::TCF7L2', 'BARX2::TEAD3', 'BARX2::UBP1',
       'BARX2::ZFHX3', 'CEBPG::NFATC4', 'CREB1::NFIA', 'CREB1::SP2',
       'CREB1::UBP1', 'CREB5::NFATC4', 'CREB5::NFIA', 'CREB5::SP2',
       'CREB5::UBP1', 'CREM::NFATC4', 'CREM::SP2', 'DDIT3::NFATC4',
       'EGR2::ELF5', 'EGR2::EPAS1', 'EGR2::HOXA10', 'EGR2::ID4',
       'EGR2::KLF4', 'EGR2::KLF5', 'EGR2::KLF9', 'EGR2::MAF',
       'EGR2::MAFG', 'EGR2::NFE2L1', 'EG

array(['ATF7::HMGA1', 'BARX2::SMARCA1', 'CEBPD::EGR2', 'EGR2::THRA',
       'EGR3::FOSL1', 'EGR3::TBX3', 'EGR3::TEAD1', 'EPAS1::SREBF2',
       'FOS::ID4', 'ID4::SNAI2', 'JUN::STAT3'], dtype=object)

In [66]:
results_stability.check_tissuetf.value_counts()

False    41
True     25
Name: check_tissuetf, dtype: int64

In [67]:
results_stability

,vocab_pair,distance,chr,start,stop,vocab1,vocab1_start,vocab1_stop,vocab2,vocab2_start,vocab2_stop,genes,tissue,check_tissuetf
0,ATF7::HMGA1,107,chr1,27481922,27482029,HMGA1,27481922,27481927,ATF7,27482019,27482029,ANKRD11|ANXA2|CTDNEP1|ELP5|OXSR1|SLC9A1|TPM3,GDSD6,False
1,ATF7::HMGA1,40,chr1,154155690,154155730,HMGA1,154155690,154155695,ATF7,154155720,154155730,ANKRD11|ANXA2|CTDNEP1|ELP5|OXSR1|SLC9A1|TPM3,GDSD6,False
2,ATF7::HMGA1,118,chr15,60685415,60685533,HMGA1,60685415,60685420,ATF7,60685523,60685533,ANKRD11|ANXA2|CTDNEP1|ELP5|OXSR1|SLC9A1|TPM3,GDSD6,False
3,ATF7::HMGA1,89,chr3,38195556,38195645,HMGA1,38195556,38195561,ATF7,38195635,38195645,ANKRD11|ANXA2|CTDNEP1|ELP5|OXSR1|SLC9A1|TPM3,GDSD6,False
4,ATF7::MAFG,30,chr1,154155700,154155730,MAFG,154155700,154155708,ATF7,154155720,154155730,SDC4|TOP1|TPM3,GDSD6,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,ID4::SNAI2,89,chr3,48471158,48471247,ID4,48471158,48471164,SNAI2,48471239,48471247,ACLY|ANAPC5|ARPC1A|ATP12A|CALML5|CCDC51|CERS3|...,GDSD6,False
62,ID4::SNAI2,54,chr7,128084571,128084625,ID4,128084571,128084577,SNAI2,128084617,128084625,ACLY|ANAPC5|ARPC1A|ATP12A|CALML5|CCDC51|CERS3|...,GDSD6,False
63,ID4::SNAI2,28,chr9,130689963,130689991,ID4,130689963,130689969,SNAI2,130689983,130689991,ACLY|ANAPC5|ARPC1A|ATP12A|CALML5|CCDC51|CERS3|...,GDSD6,False
64,JUN::STAT3,56,chr20,33532191,33532247,STAT3,33532191,33532200,JUN,33532239,33532247,ARL13B|ERP44|GSS,GDSD6,True
